In [1]:
import os
import sys
import shutil
import random
import time
PROJ_DIR = os.environ["RNB_PLANNING_DIR"]
sys.path.append(os.path.join(PROJ_DIR, "src"))

import SharedArray as sa
import numpy as np
import time

from pkg.utils.utils import *

args = DummyObject()
args.model_path = "None"
args.rtype="panda"
args.precision="FP16"

if args.model_path == "None":
    args.model_path = None
PRECISION = args.precision

import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2


from pkg.utils.utils_python3 import *
from pkg.controller.robot_config import RobotType
from pkg.planning.filtering.lattice_model.data_utils import *
import numpy as np
int2rtypename = {v.value:v.name for v in RobotType}
DATA_PATH = os.path.join(PROJ_DIR, "data")
MODEL_PATH = os.path.join(PROJ_DIR, "model")
LAT_MODEL_PATH = os.path.join(MODEL_PATH,"latticized")
try_mkdir(MODEL_PATH)
try_mkdir(LAT_MODEL_PATH)
GRASP_FOLDER = "grasp"
ARM10_FOLDER = "arm_10"
ARM05_FOLDER = "arm_05"
FULLS_FOLDER = "full_scene"

ARM_FOLDER = ARM10_FOLDER
GRASP_SHAPE = (20,20,20)
ARM_SHAPE = (20,20,20)
RH_MASK_SIZE = 512
RH_MASK_STEP = 64

BATCH_SIZE = 1
SERVER_PERIOD = 1e-3

In [2]:
self, prepared_p = DummyObject, sa.create(f"shm://{args.rtype}.prepared", (1,), dtype=np.bool)
self.ROBOT_TYPE_NAME=args.rtype

grasp_img_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.grasp_img", (BATCH_SIZE,) + GRASP_SHAPE + (3,))
arm_img_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.arm_img", (BATCH_SIZE,) + ARM_SHAPE + (1,))
rh_vals_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.rh_vals", (BATCH_SIZE, 2))
result_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.result", (BATCH_SIZE, 2))
query_in = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.query_in", (1,), dtype=np.bool)
response_out = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.response_out", (1,), dtype=np.bool)
query_quit = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.query_quit", (1,), dtype=np.bool)
gtimer = GlobalTimer.instance()
gtimer.reset()
grasp_img_p[:] = 0
arm_img_p[:] = 0
rh_vals_p[:] = 0
result_p[:] = 0
query_in[0] = False
response_out[0] = False
query_quit[0] = False
rh_mask = np.zeros((BATCH_SIZE, 54))

In [3]:
ROBOT_TYPE_NAME=self.ROBOT_TYPE_NAME
ROBOT_MODEL_ROOT = os.path.join(LAT_MODEL_PATH, ROBOT_TYPE_NAME)
last_model = sorted(os.listdir(ROBOT_MODEL_ROOT))[-1]
last_save = sorted([item for item in os.listdir(os.path.join(ROBOT_MODEL_ROOT, last_model)) if item.startswith("model")])[-1]
model_path_rel = os.path.join(last_model, last_save)
model_log_dir = os.path.join(ROBOT_MODEL_ROOT, model_path_rel)
model_log_dir_trt = os.path.join(ROBOT_MODEL_ROOT, model_path_rel.replace("model", "trt")+"-"+PRECISION)

### Load saved converted model

In [4]:
saved_model_loaded = tf.saved_model.load(
    model_log_dir_trt, tags=[tag_constants.SERVING])
graph_func = saved_model_loaded.signatures[
    signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY]

In [5]:
@tf.function
def inference(grasp_img_t, arm_img_t, rh_mask_t):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = saved_model_loaded([grasp_img_t, arm_img_t, rh_mask_t])
    return predictions

### convert_variables_to_constants_v2

In [6]:
from tensorflow.python.framework import convert_to_constants

In [7]:
func = graph_func
lower_control_flow=True
aggressive_inlining=False


converter_data = convert_to_constants._FunctionConverterDataInEager(
      func=func,
      lower_control_flow=lower_control_flow,
      aggressive_inlining=aggressive_inlining)
output_graph_def, converted_input_indices = convert_to_constants._replace_variables_by_constants(
      converter_data=converter_data)

fronzen_func = convert_to_constants._construct_concrete_function(func, output_graph_def,
                                                                 converted_input_indices)

In [9]:
gtimer.tic("convert_tf")
query_in[0] = False
## TODO: inference depending on robot type
r_mask = div_r_gaussian(rh_vals_p[0][0])
h_mask = div_h_gaussian(rh_vals_p[0][1])
rh_mask[0] = np.concatenate([r_mask, h_mask])
grasp_img_t, arm_img_t, rh_mask_t = (
    tf.constant(grasp_img_p, dtype=tf.float32), 
    tf.constant(arm_img_p, dtype=tf.float32), 
    tf.constant(rh_mask, dtype=tf.float32))
etime_ctf = gtimer.toc("convert_tf")
print("convertin : {} ms".format(round(etime_ctf, 2)))
gtimer.tic("inference_0")
result_inf = inference(grasp_img_t, arm_img_t, rh_mask_t)
etime_inf = gtimer.toc("inference_0")
print("inference : {} ms".format(round(etime_inf, 2)))
gtimer.tic("inference_1")
result_frz = fronzen_func(grasp_img_t, arm_img_t, rh_mask_t)
etime_inf = gtimer.toc("inference_1")
print("frozen inf : {} ms".format(round(etime_inf, 2)))
for i_b in range(BATCH_SIZE):
    result_p[i_b] = result_inf[i_b]
response_out[0] = True

convertin : 1.45 ms
inference : 30.47 ms
frozen inf : 10.88 ms


In [44]:
gtimer=GlobalTimer.instance()
gtimer.reset()
while not query_quit[0]:
    if not query_in[0]:
        time.sleep(SERVER_PERIOD)
        continue
    gtimer.tic("convert_tf")
    query_in[0] = False
    ## TODO: inference depending on robot type
    r_mask = div_r_gaussian(rh_vals_p[0][0])
    h_mask = div_h_gaussian(rh_vals_p[0][1])
    rh_mask[0] = np.concatenate([r_mask, h_mask])
    grasp_img_t, arm_img_t, rh_mask_t = (
        tf.constant(grasp_img_p, dtype=tf.float32), 
        tf.constant(arm_img_p, dtype=tf.float32), 
        tf.constant(rh_mask, dtype=tf.float32))
    etime_ctf = gtimer.toc("convert_tf")
    print("convertin : {} ms".format(round(etime_ctf, 2)))
    gtimer.tic("inference_0")
    result_inf = inference(grasp_img_t, arm_img_t, rh_mask_t)
    etime_inf = gtimer.toc("inference_0")
    print("inference : {} ms".format(round(etime_inf, 2)))
    gtimer.tic("inference_1")
    result_frz = fronzen_func(grasp_img_t, arm_img_t, rh_mask_t)
    etime_inf = gtimer.toc("inference_1")
    print("frozen inf : {} ms".format(round(etime_inf, 2)))
    for i_b in range(BATCH_SIZE):
        result_p[i_b] = result_inf[i_b]
    response_out[0] = True

convertin : 0.22 ms
inference : 45.58 ms
convertin : 0.18 ms
inference : 35.05 ms
convertin : 0.19 ms
inference : 34.7 ms
convertin : 0.17 ms
inference : 19.09 ms
convertin : 0.17 ms
inference : 22.58 ms
convertin : 0.2 ms
inference : 21.32 ms
convertin : 0.17 ms
inference : 34.26 ms
convertin : 0.2 ms
inference : 30.9 ms
convertin : 0.18 ms
inference : 29.55 ms
convertin : 0.18 ms
inference : 36.96 ms
convertin : 0.19 ms
inference : 37.66 ms
convertin : 0.2 ms
inference : 34.23 ms
convertin : 0.18 ms
inference : 39.5 ms
convertin : 0.19 ms
inference : 33.8 ms
convertin : 0.21 ms
inference : 20.37 ms
convertin : 0.19 ms
inference : 38.65 ms
convertin : 0.19 ms
inference : 20.37 ms
convertin : 0.19 ms
inference : 34.66 ms
convertin : 0.18 ms
inference : 37.12 ms
convertin : 0.19 ms
inference : 34.24 ms
convertin : 0.18 ms
inference : 34.06 ms
convertin : 0.18 ms
inference : 33.27 ms
convertin : 0.18 ms
inference : 29.72 ms
convertin : 0.18 ms
inference : 35.87 ms
convertin : 0.1 ms
infe

In [45]:
print(gtimer)

convert_tf: 	18.0 ms/100 = 0.182 ms (0.09/0.222)
inference: 	3184.0 ms/100 = 31.837 ms (18.843/55.542)



In [3]:
self = DummyObject()
self.ROBOT_TYPE_NAME = "panda"
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.grasp_img")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.arm_img")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.rh_vals")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.result")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.query_in")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.response_out")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.query_quit")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.prepared")